In [2]:
import os
import json
import http.client
import requests
from typing import Dict, List, Optional, Union, Any
from dataclasses import dataclass
from enum import Enum
import re
from dotenv import load_dotenv
from pydantic import BaseModel, Field

# LangChain imports for @tool decorator
from langchain.tools import tool
from langchain_core.tools import BaseTool
from langchain.schema import BaseMessage

# Load environment variables
load_dotenv()

In [3]:
class ConfidenceLevel(Enum):
    HIGH = "high"
    MEDIUM = "medium"
    LOW = "low"

@dataclass
class QueryAnalysis:
    confidence: ConfidenceLevel
    reasoning: str
    requires_web_search: bool
    search_query: Optional[str] = None

class SearchInput(BaseModel):
    """Input schema for search tools"""
    query: str = Field(description="The search query string")
    country: str = Field(default="us", description="Country code for localized results")

class WeatherInput(BaseModel):
    """Input schema for weather tool"""
    location: str = Field(description="City name for weather information")

In [4]:
@tool
def search_web_serper(query: str, country: str = "us") -> str:
    """
    Search the web using Serper API for current information.
    Use this for current events, prices, news, and real-time data.

    Args:
        query: The search query
        country: Country code for localized results (default: us)

    Returns:
        Formatted search results as string
    """
    api_key = os.getenv("SERPER_API_KEY")
    if not api_key:
        return "❌ Error: SERPER_API_KEY not found in environment variables"

    try:
        conn = http.client.HTTPSConnection("google.serper.dev")
        payload = json.dumps({
            "q": query,
            "gl": country,
            "num": 5
        })
        headers = {
            'X-API-KEY': api_key,
            'Content-Type': 'application/json'
        }

        conn.request("POST", "/search", payload, headers)
        res = conn.getresponse()
        data = res.read()
        conn.close()

        results = json.loads(data.decode("utf-8"))

        if "organic" not in results:
            return "❌ No search results found"

        # Format results
        formatted = "🔍 **Web Search Results:**\n\n"
        for i, result in enumerate(results["organic"][:3], 1):
            title = result.get("title", "No title")
            snippet = result.get("snippet", "No description")
            link = result.get("link", "")

            formatted += f"**{i}. {title}**\n"
            formatted += f"{snippet}\n"
            formatted += f"🔗 {link}\n\n"

        return formatted

    except Exception as e:
        return f"❌ Search error: {str(e)}"

@tool
def search_web_duckduckgo(query: str) -> str:
    """
    Search the web using DuckDuckGo (free, no API key required).
    Good for general web searches and privacy-conscious queries.

    Args:
        query: The search query

    Returns:
        Search results as formatted string
    """
    try:
        # Simple DuckDuckGo search using requests
        url = "https://api.duckduckgo.com/"
        params = {
            'q': query,
            'format': 'json',
            'no_redirect': '1',
            'no_html': '1',
            'skip_disambig': '1'
        }

        response = requests.get(url, params=params, timeout=10)
        data = response.json()

        # Format results
        formatted = "🦆 **DuckDuckGo Search Results:**\n\n"

        # Abstract (main answer)
        if data.get('Abstract'):
            formatted += f"**Summary:** {data['Abstract']}\n"
            if data.get('AbstractURL'):
                formatted += f"🔗 Source: {data['AbstractURL']}\n\n"

        # Related topics
        if data.get('RelatedTopics'):
            formatted += "**Related Information:**\n"
            for i, topic in enumerate(data['RelatedTopics'][:3], 1):
                if isinstance(topic, dict) and 'Text' in topic:
                    formatted += f"{i}. {topic['Text']}\n"
                    if topic.get('FirstURL'):
                        formatted += f"   🔗 {topic['FirstURL']}\n"
            formatted += "\n"

        # If no good results, indicate that
        if not data.get('Abstract') and not data.get('RelatedTopics'):
            formatted += "Limited results found. Try a more specific query.\n"

        return formatted

    except Exception as e:
        return f"❌ DuckDuckGo search error: {str(e)}"

@tool
def get_current_bitcoin_price() -> str:
    """
    Get the current Bitcoin price in USD.
    Specialized tool for cryptocurrency price information.

    Returns:
        Current Bitcoin price and market data
    """
    try:
        # Using a free cryptocurrency API
        url = "https://api.coindesk.com/v1/bpi/currentprice/USD.json"
        response = requests.get(url, timeout=10)
        data = response.json()

        price = data['bpi']['USD']['rate']
        last_updated = data['time']['updated']

        result = f"₿ **Current Bitcoin Price:**\n\n"
        result += f"**Price:** ${price}\n"
        result += f"**Last Updated:** {last_updated}\n"
        result += f"**Source:** CoinDesk Bitcoin Price Index\n"

        return result

    except Exception as e:
        return f"❌ Bitcoin price fetch error: {str(e)}"

@tool
def get_weather_info(location: str) -> str:
    """
    Get current weather information for a specific location.

    Args:
        location: City name or location

    Returns:
        Current weather information
    """
    try:
        # Using a free weather API (OpenWeatherMap free tier)
        # Note: In production, you'd want to use a proper API key
        # This is a placeholder implementation

        # For demo purposes, return mock weather data
        result = f"🌤️ **Weather in {location}:**\n\n"
        result += "**Temperature:** 22°C (72°F)\n"
        result += "**Condition:** Partly Cloudy\n"
        result += "**Humidity:** 65%\n"
        result += "**Note:** This is demo data. Add OpenWeatherMap API key for real data.\n"

        return result

    except Exception as e:
        return f"❌ Weather fetch error: {str(e)}"

@tool
def analyze_query_confidence_tool(query: str) -> str:
    """
    Analyze a query to determine confidence level and search necessity.
    This is a meta-tool that helps decide which other tools to use.

    Args:
        query: The user query to analyze

    Returns:
        Analysis of confidence level and recommendations
    """
    agent = WebBrowsingAgent()  # We'll define this class below
    analysis = agent.analyze_query_confidence(query)

    result = f"🎯 **Query Analysis:**\n\n"
    result += f"**Query:** {query}\n"
    result += f"**Confidence Level:** {analysis.confidence.value.upper()}\n"
    result += f"**Reasoning:** {analysis.reasoning}\n"
    result += f"**Web Search Needed:** {'Yes' if analysis.requires_web_search else 'No'}\n"

    if analysis.requires_web_search:
        result += f"**Optimized Search Query:** {analysis.search_query}\n"
        result += f"**Recommended Action:** Use web search tools\n"
    else:
        result += f"**Recommended Action:** Use existing knowledge\n"

    return result


In [5]:
class WebBrowsingAgent:
    """
    Web browsing agent that uses @tool decorated functions
    """

    def __init__(self):
        # Register all available tools
        self.tools = {
            'search_web_serper': search_web_serper,
            'search_web_duckduckgo': search_web_duckduckgo,
            'get_current_bitcoin_price': get_current_bitcoin_price,
            'get_weather_info': get_weather_info,
            'analyze_query_confidence_tool': analyze_query_confidence_tool
        }

        # Confidence analysis parameters
        self.current_info_keywords = [
            'current', 'latest', 'recent', 'now', 'today', 'this year',
            'price', 'stock', 'weather', 'news', 'trending', 'live'
        ]

        self.dynamic_topics = [
            'bitcoin', 'cryptocurrency', 'stock market', 'weather', 'news',
            'politics', 'sports scores', 'exchange rates'
        ]

        self.factual_indicators = [
            'capital of', 'formula for', 'definition of', 'how to calculate',
            'what does', 'explain', 'difference between', 'history of',
            'located in', 'known for', 'famous for', 'invented by'
        ]

    def analyze_query_confidence(self, query: str) -> QueryAnalysis:
        """Analyze query confidence (same logic as before)"""
        query_lower = query.lower()

        # Check for current information needs
        has_current_keywords = any(keyword in query_lower for keyword in self.current_info_keywords)
        has_dynamic_topics = any(topic in query_lower for topic in self.dynamic_topics)
        has_recent_dates = bool(re.search(r'\b(202[4-9]|20[3-9]\d)\b', query))

        # Check for factual information
        has_factual_indicators = any(indicator in query_lower for indicator in self.factual_indicators)
        is_math_science = any(term in query_lower for term in [
            'calculate', 'formula', 'equation', 'theorem', 'definition'
        ])

        # Decision logic
        if has_current_keywords or has_recent_dates or has_dynamic_topics:
            confidence = ConfidenceLevel.LOW
            reasoning = "Query requires current/real-time information"
            requires_search = True
        elif has_factual_indicators or is_math_science:
            confidence = ConfidenceLevel.HIGH
            reasoning = "Query involves well-established factual information"
            requires_search = False
        elif 'who is' in query_lower:
            confidence = ConfidenceLevel.MEDIUM
            reasoning = "Query about specific person that may not be in training data"
            requires_search = True
        else:
            confidence = ConfidenceLevel.MEDIUM
            reasoning = "Uncertain about completeness of available information"
            requires_search = True

        # Generate search query
        search_query = self._optimize_search_query(query) if requires_search else None

        return QueryAnalysis(
            confidence=confidence,
            reasoning=reasoning,
            requires_web_search=requires_search,
            search_query=search_query
        )

    def _optimize_search_query(self, query: str) -> str:
        """Optimize query for search"""
        stop_words = ['what', 'who', 'when', 'where', 'why', 'how', 'is', 'are', 'the', 'a', 'an']
        words = query.lower().split()
        filtered_words = [word.rstrip('?') for word in words if word.rstrip('?') not in stop_words]
        return ' '.join(filtered_words[:6])

    def select_appropriate_tool(self, query: str, analysis: QueryAnalysis) -> str:
        """Select the most appropriate tool based on query analysis"""
        query_lower = query.lower()

        # Special cases for specific tools
        if 'bitcoin' in query_lower and 'price' in query_lower:
            return 'get_current_bitcoin_price'
        elif 'weather' in query_lower:
            return 'get_weather_info'
        elif analysis.confidence == ConfidenceLevel.LOW:
            # For current info, prefer Serper if available, else DuckDuckGo
            return 'search_web_serper' if os.getenv("SERPER_API_KEY") else 'search_web_duckduckgo'
        else:
            # Default to DuckDuckGo for general searches
            return 'search_web_duckduckgo'

    def answer_query(self, query: str, base_knowledge: str = None) -> Dict:
        """Main method to answer queries using @tool functions"""

        # Analyze the query
        analysis = self.analyze_query_confidence(query)

        response = {
            "query": query,
            "confidence_analysis": {
                "level": analysis.confidence.value,
                "reasoning": analysis.reasoning,
                "web_search_used": analysis.requires_web_search
            },
            "available_tools": list(self.tools.keys())
        }

        if not analysis.requires_web_search:
            # High confidence - use base knowledge
            response["answer"] = base_knowledge or "Based on my knowledge: [Answer would be provided here]"
            response["sources"] = "Internal knowledge"
            response["tool_used"] = None

        else:
            # Select and use appropriate tool
            selected_tool_name = self.select_appropriate_tool(query, analysis)
            selected_tool = self.tools[selected_tool_name]

            try:
                # Call the @tool function
                if selected_tool_name == 'get_weather_info':
                    # Extract location from query
                    location = query.lower().replace('weather', '').replace('in', '').strip()
                    if not location:
                        location = "New York"  # Default
                    tool_result = selected_tool.invoke({"location": location})
                else:
                    # For search tools, use the optimized query
                    search_query = analysis.search_query or query
                    tool_result = selected_tool.invoke({"query": search_query})

                # Combine with base knowledge if available
                if base_knowledge:
                    response["answer"] = f"{base_knowledge}\n\n{tool_result}"
                else:
                    response["answer"] = tool_result

                response["sources"] = f"@tool function: {selected_tool_name}"
                response["tool_used"] = selected_tool_name
                response["search_query_used"] = analysis.search_query

            except Exception as e:
                response["answer"] = f"❌ Tool execution error: {str(e)}"
                response["sources"] = "Error"
                response["tool_used"] = selected_tool_name

        return response


In [6]:
def main():
    """Test the @tool based web browsing agent"""

    print("🛠️ LangChain @tool Decorator Web Browsing Agent")
    print("=" * 60)

    # Initialize agent
    agent = WebBrowsingAgent()

    print("Available @tool functions:")
    for tool_name in agent.tools.keys():
        print(f"  📦 {tool_name}")
    print("=" * 60)

    # Test queries
    test_queries = [
        "What is the capital of France?",        # HIGH - no search
        "Current Bitcoin price",                 # LOW - special Bitcoin tool
        "Who is Ojasw Kant?",                   # MEDIUM - web search
        "Weather in New York today",           # LOW - weather tool
        "Latest AI news",                       # LOW - web search
        "How to calculate compound interest",    # HIGH - no search
    ]

    for i, query in enumerate(test_queries, 1):
        print(f"\n{i}. Query: '{query}'")

        # Analyze confidence first
        analysis = agent.analyze_query_confidence(query)
        print(f"   🎯 Confidence: {analysis.confidence.value}")
        print(f"   💭 Reasoning: {analysis.reasoning}")
        print(f"   🔍 Search needed: {analysis.requires_web_search}")

        if analysis.requires_web_search:
            selected_tool = agent.select_appropriate_tool(query, analysis)
            print(f"   🛠️  Selected tool: {selected_tool}")
            print(f"   🔎 Search query: {analysis.search_query}")

        # Uncomment to test actual tool execution
        # result = agent.answer_query(query)
        # print(f"   📝 Tool used: {result.get('tool_used', 'None')}")
        # print(f"   📋 Answer preview: {result['answer'][:100]}...")

        print("-" * 60)

def test_individual_tools():
    """Test individual @tool functions"""
    print("\n🧪 Testing Individual @tool Functions:")
    print("=" * 50)

    # Test Bitcoin price tool
    print("\n1. Testing Bitcoin Price Tool:")
    try:
        result = get_current_bitcoin_price.invoke({})
        print(result[:200] + "..." if len(result) > 200 else result)
    except Exception as e:
        print(f"❌ Error: {e}")

    # Test DuckDuckGo search
    print("\n2. Testing DuckDuckGo Search Tool:")
    try:
        result = search_web_duckduckgo.invoke({"query": "Python programming"})
        print(result[:200] + "..." if len(result) > 200 else result)
    except Exception as e:
        print(f"❌ Error: {e}")

    # Test confidence analysis tool
    print("\n3. Testing Confidence Analysis Tool:")
    try:
        result = analyze_query_confidence_tool.invoke({"query": "What is the capital of France?"})
        print(result)
    except Exception as e:
        print(f"❌ Error: {e}")

if __name__ == "__main__":
    main()

    # Uncomment to test individual tools
    # test_individual_tools()

=== Web Browsing Agent Test ===

Query: What is the capital of France?
Confidence: medium
Reasoning: Query contains specific entities that may not be in training data
Web search needed: True
Search query: capital of france?
--------------------------------------------------
Query: Current price of Bitcoin
Confidence: low
Reasoning: Query contains current/recent information indicators
Web search needed: True
Search query: current price of bitcoin
--------------------------------------------------
Query: Who is Ojasw Kant?
Confidence: medium
Reasoning: Query contains specific entities that may not be in training data
Web search needed: True
Search query: ojasw kant?
--------------------------------------------------
Query: Latest news about AI
Confidence: low
Reasoning: Query contains current/recent information indicators
Web search needed: True
Search query: latest news about ai
--------------------------------------------------
Query: How to calculate compound interest
Confidence: high